In [ ]:

# CELL 1 - Imports
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split


In [ ]:

image_path = "C:\Users\Lizbel\OneDrive\Pictures"  # <- replace with your image path
image = cv2.imread(image_path)

if image is None:
    raise FileNotFoundError("Make sure the image path is correct.")

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)
plt.title("Custom Image")
plt.axis("off")
plt.show()



error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\utils\samples.cpp:64: error: (-2:Unspecified error) OpenCV samples: Can't find required data file: lena.jpg in function 'cv::samples::findFile'


In [ ]:

# CELL 4 - Create Dummy Shape Dataset
def create_dummy_image(label):
    img = np.zeros((64, 64), dtype=np.uint8)
    if label == 0:
        cv2.circle(img, (32, 32), 20, 255, -1)
    else:
        cv2.rectangle(img, (20, 20), (44, 44), 255, -1)
    return img

X = []
y = []

for label in [0, 1]:
    for _ in range(300):
        X.append(create_dummy_image(label))
        y.append(label)

X = np.array(X).astype(np.float32) / 255.0
y = np.array(y)
X = X.reshape(-1, 1, 64, 64)


In [ ]:

# CELL 5 - Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)


In [ ]:

# CELL 6 - Define Neural Network
class ShapeClassifier(nn.Module):
    def __init__(self):
        super(ShapeClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*64, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.net(x)

model = ShapeClassifier()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:

# CELL 7 - Train the Model
epochs = 10

for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        out = model(batch_X)
        loss = loss_fn(out, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


In [ ]:

# CELL 8 - Evaluate Model
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor)
    y_pred = torch.argmax(preds, dim=1).numpy()

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Confusion Matrix:\n{cm}")


In [ ]:

# CELL 9 - Save Model as .pth
torch.save(model.state_dict(), "model_weights.pth")
print("Model saved as model_weights.pth")


In [ ]:

# CELL 10 - Load Model
model_loaded = ShapeClassifier()
model_loaded.load_state_dict(torch.load("model_weights.pth"))
model_loaded.eval()
print("Model loaded successfully!")
